In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import time
from splinter import Browser
import warnings
warnings.filterwarnings('ignore')
print('Libraries imported!')

Libraries imported!


In [2]:
house_address = []
house_details = []
PostalCode = []
Latitude = []
Longitude = []
YearBuilt = []

base_url = 'https://www.remax.ca/ab/calgary-real-estate?page='
urls = [base_url + str(x) for x in range(1,2)]

for url in urls:
    
    #browser.visit(url)
    time.sleep(5)

    # Parse HTML with Beautiful Soup
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    time.sleep(5)

    try:
        addresses = soup.find_all('div', class_='left-content flex-one')
        for address in addresses:
            house_address.append(address.text)
    except:
        house_address.append('None')
        
    time.sleep(5)
        
    try:
        details = soup.find_all('div', class_='property-details')
        for detail in details:
            house_details.append(detail.text)
    except:
        house_details.append('None')
        
    time.sleep(5)
        
    house_details_more=[]
    url_house = []
        
    try:
        for detail in soup.find_all('a'):
            house_details_more.append(detail.get('href'))
    except:
        house_details_more.append('None')
    
    time.sleep(5)

    for i in house_details_more:
        if '/ab/calgary-real-estate/' in str(i):
            link = 'https://www.remax.ca' + str(i)
            url_house.append(link)
            
    time.sleep(5)
    
    for i in url_house[:10]:
    
        time.sleep(5)

        page = requests.get(i)
        soup = BeautifulSoup(page.text, "html.parser")

        lat_all = soup.find_all('script')[4]

        string = ', '.join(lat_all)
        pc = "postalCode"
        position_pc = string.find(pc)
        pcd = string[(position_pc+14):(position_pc+21)]
        PostalCode.append(pcd)

        lat = "latitude"
        position_lat = string.find(lat)
        latd = string[(position_lat+11):(position_lat+20)]
        Latitude.append(latd)

        long = "longitude"
        position_long = string.find(long)
        longd = string[(position_long+12):(position_long+23)]
        Longitude.append(longd)

        house_address = []

        time.sleep(5)

        try:
            addresses = soup.find_all('section', class_='property-details')
            for address in addresses:
                house_address.append(address.text)
        except:
            house_address.append('None')

        time.sleep(5)

        string = ', '.join(house_address)
        s_year = "Year Built"
        position_year = string.find(s_year)
        year_built = string[(position_year+12):(position_year+16)]
        YearBuilt.append(year_built)

        time.sleep(5)

In [6]:
#PostalCode
#Latitude
#Longitude
#YearBuilt

['1977',
 '1964',
 '1974',
 '2011',
 '2010',
 '1955',
 '1999',
 '1976',
 '1999',
 '1999']

In [7]:
address_df = pd.DataFrame(house_address)

new_df = address_df[0].str.split(' ', 2, expand=True)
new_df["price"] = new_df[1].str.replace("$", "")
new_df["price"] = new_df["price"].str.replace(",", "")

del new_df[0]
del new_df[1]
new_df.head()

,2,price
0,97 Prestwick Gardens SEProperty Type: Townhome...,for


-----------

<div class="alert alert-info">
    <b>UNSED</b>
</div>

In [6]:
house_details_more=[]
url_house = []
try:
    for detail in soup.find_all('a'):
        house_details_more.append(detail.get('href'))
except:
    house_details_more.append('None')
    
for i in house_details_more:
    if '/ab/calgary-real-estate/' in str(i):
        link = 'https://www.remax.ca' + str(i)
        url_house.append(link)
url_house

['https://www.remax.ca/ab/calgary-real-estate/4312-49-st-ne-wp_id282699451-lst',
 'https://www.remax.ca/ab/calgary-real-estate/3916-vancouver-crescent-nw-wp_id282833251-lst',
 'https://www.remax.ca/ab/calgary-real-estate/102-99-arbour-lake-road-nw-wp_id282825809-lst',
 'https://www.remax.ca/ab/calgary-real-estate/63-cranarch-circle-se-wp_id282826543-lst',
 'https://www.remax.ca/ab/calgary-real-estate/32-copperpond-close-se-wp_id282827182-lst',
 'https://www.remax.ca/ab/calgary-real-estate/1408-43-street-sw-wp_id282826546-lst',
 'https://www.remax.ca/ab/calgary-real-estate/1314-7451-springbank-boulevard-sw-wp_id282825813-lst',
 'https://www.remax.ca/ab/calgary-real-estate/54-6103-madigan-drive-ne-wp_id282827181-lst',
 'https://www.remax.ca/ab/calgary-real-estate/303-4-14-street-nw-wp_id282825548-lst',
 'https://www.remax.ca/ab/calgary-real-estate/97-prestwick-gardens-se-wp_id282825546-lst',
 'https://www.remax.ca/luxury/ab/calgary-real-estate/707-35-street-nw-wp_id282831352-lst',
 'http

In [16]:
url_house[:5]

['https://www.remax.ca/ab/calgary-real-estate/4312-49-st-ne-wp_id282699451-lst',
 'https://www.remax.ca/ab/calgary-real-estate/3916-vancouver-crescent-nw-wp_id282833251-lst',
 'https://www.remax.ca/ab/calgary-real-estate/102-99-arbour-lake-road-nw-wp_id282825809-lst',
 'https://www.remax.ca/ab/calgary-real-estate/63-cranarch-circle-se-wp_id282826543-lst',
 'https://www.remax.ca/ab/calgary-real-estate/32-copperpond-close-se-wp_id282827182-lst']

In [26]:
PostalCode = []
Latitude = []
Longitude = []
YearBuilt = []

for i in url_house[:5]:
    
    time.sleep(5)
    
    page = requests.get(i)
    soup = BeautifulSoup(page.text, "html.parser")
    
    lat_all = soup.find_all('script')[4]
    
    string = ', '.join(lat_all)
    pc = "postalCode"
    position_pc = string.find(pc)
    pcd = string[(position_pc+14):(position_pc+21)]
    PostalCode.append(pcd)

    lat = "latitude"
    position_lat = string.find(lat)
    latd = string[(position_lat+11):(position_lat+20)]
    Latitude.append(latd)

    long = "longitude"
    position_long = string.find(long)
    longd = string[(position_long+12):(position_long+23)]
    Longitude.append(longd)
    
    house_address = []
    
    time.sleep(5)
    
    try:
        addresses = soup.find_all('section', class_='property-details')
        for address in addresses:
            house_address.append(address.text)
    except:
        house_address.append('None')
    
    time.sleep(5)
    
    string = ', '.join(house_address)
    s_year = "Year Built"
    position_year = string.find(s_year)
    year_built = string[(position_year+12):(position_year+16)]
    YearBuilt.append(year_built)